# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order according to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
def date_sorter():
    
    # case 1: 04/20/2009; 04/20/09; 4/20/09; 4/3/09
    case1 = df.str.extractall(r"(\d\d?)[/-](\d\d?)[/-]([12][019]\d\d|\d\d)\b")
    case1.columns = ["month", "day", "year"]
    case1["month"].loc[case1["month"].str.len() == 1] = "0"+case1["month"]
    case1["day"].loc[case1["day"].str.len() == 1] = "0"+case1["day"]
    case1["year"].loc[case1["year"].str.len() == 2] = "19"+case1["year"]
    case1.reset_index(inplace = True)
    case1.set_index("level_0", inplace = True)
    case1 = case1[["year", "month", "day"]]
    
    month_dict = {"Jan":"01", "Feb":"02", "Mar":"03", "Apr":"04", "May":"05", "Jun":"06",
              "Jul":"07", "Aug":"08", "Sep":"09", "Oct":"10", "Nov":"11", "Dec":"12"}
    
    # case 2: Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
    case2 = df.str.extractall(r"(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.? (\d\d),? (\d\d\d\d)\b")
    case2.columns = ["month", "day", "year"]
    case2.reset_index(inplace = True)
    case2.set_index("level_0", inplace = True)
    case2["month"].replace(month_dict, inplace = True)
    case2 = case2[["year", "month", "day"]]
    
    # case 3: 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    case3 = df.str.extractall(r"(\d\d) (Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\.?,? (\d\d\d\d)\b")
    case3.columns = ["day", "month", "year"]
    case3.reset_index(inplace = True)
    case3.set_index("level_0", inplace = True)
    case3["month"].replace(month_dict, inplace = True)
    case3 = case3[["year", "month", "day"]]

    # case 4: Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009 - no lines match

    # case 5: Feb 2009; Sep 2009; Oct 2010
    case5 = df.str.extractall(r"(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,? (\d\d\d\d)\b")
    case5["day"] = "01"
    case5.columns = ["month", "year", "day"]
    case5.reset_index(inplace = True)
    case5.set_index("level_0", inplace = True)
    case5["month"].replace(month_dict, inplace = True)
    case5 = case5[["year", "month", "day"]]

    # case 6: 6/2008; 12/2009
    case6 = df.str.extractall(r"(\d\d?)/(\d\d\d\d)\b")
    case6["day"] = "01"
    case6.columns = ["month", "year", "day"]
    case6.reset_index(inplace = True)
    case6.set_index("level_0", inplace = True)
    case6["month"].loc[case6["month"].str.len() == 1] = "0"+case6["month"]
    case6 = case6[["year", "month", "day"]]

    # case 7: 2009; 2010
    case7 = df.str.extractall(r"(20\d\d|19\d\d)")
    case7["month"] = "01"
    case7["day"] = "01"
    case7.columns = ["year", "month", "day"]
    case7.reset_index(inplace = True)
    case7.set_index("level_0", inplace = True)
    case7 = case7[["year", "month", "day"]]

    # concatenate cases
    x = pd.concat([case1, case2.loc[case2.index.isin(case1.index) == False]], verify_integrity = True)
    x = pd.concat([x, case3.loc[case3.index.isin(x.index) == False]], verify_integrity = True)
    x = pd.concat([x, case5.loc[case5.index.isin(x.index) == False]], verify_integrity = True)
    x = pd.concat([x, case6.loc[case6.index.isin(x.index) == False]], verify_integrity = True)
    x = pd.concat([x, case7.loc[case7.index.isin(x.index) == False]], verify_integrity = True)

    x.sort_values(by = ["year", "month", "day"], inplace = True)
    x.reset_index(inplace = True)
    answer = x["level_0"]

    return answer

In [4]:
print(date_sorter())

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    286
478    480
479    431
480    279
481    198
482    381
483    463
484    366
485    255
486    439
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Name: level_0, Length: 500, dtype: int64
